<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/sparsity_sorted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.manual_seed(1234)
np.random.seed(1234)

In [4]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

In [6]:
new_mnist_trainset =  [ [[],[]] for i in range(10)]
new_mnist_testset  =  [ [[],[]] for i in range(10)]

for i in range(60000):
    for j in range(10):
        if mnist_trainset[i][1] == j:
            # image 
            new_mnist_trainset[j][0].append(mnist_trainset[i][0])  
            # label
            new_mnist_trainset[j][1].append(mnist_trainset[i][1])

for i in range(10000):
    for j in range(10):
        if mnist_testset[i][1] == j:
            # image 
            new_mnist_testset[j][0].append(mnist_testset[i][0])  
            # label
            new_mnist_testset[j][1].append(mnist_testset[i][1])

image_trainset = list()
label_trainset = list()

image_testset = list()
label_testset = list()

for i in range(10):
    image_trainset.append(new_mnist_trainset[i][0])
    label_trainset.append(new_mnist_trainset[i][1])

for i in range(10):
    image_testset.append(new_mnist_testset[i][0])
    label_testset.append(new_mnist_testset[i][1])

flattened_image_train = list()
flattened_label_train = list()

flattened_image_test = list()
flattened_label_test = list()

# flattening image 
for sublist in image_trainset:
    for val in sublist:
        flattened_image_train.append(val)

# flattening label
for sublist in label_trainset:
    for val in sublist:
        flattened_label_train.append(val)

# flattening image 
for sublist in image_testset:
    for val in sublist:
        flattened_image_test.append(val)

# flattening label
for sublist in label_testset:
    for val in sublist:
        flattened_label_test.append(val)

flattened_image_train = torch.stack(flattened_image_train)
flattened_label_train = torch.Tensor(flattened_label_train)
flattened_label_train = flattened_label_train.type(torch.LongTensor)

flattened_image_test = torch.stack(flattened_image_test)
flattened_label_test = torch.Tensor(flattened_label_test)
flattened_label_test = flattened_label_test.type(torch.LongTensor)

train_dataset = TensorDataset(flattened_image_train, flattened_label_train)
train_dataloader = DataLoader(train_dataset, batch_size=50)

test_dataset = TensorDataset(flattened_image_test, flattened_label_test)
test_dataloader = DataLoader(test_dataset, batch_size=50)

In [7]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid  = torch.nn.Sigmoid()

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        pred = self.linear_2(x)

        return pred

In [8]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [9]:
no_epochs = 100
criterion = torch.nn.CrossEntropyLoss()
def sparsity_trainer(optimizer, model):

    # reset the model 
    train_loss = list()
    test_loss  = list()
    test_acc   = list()
    final_spareness = list()

    # define activation list 

    best_test_loss = 1

    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        hidden_layer_activation_list = list()

        # training
        # set up training mode 
        model.train()
        for itr, (image, label) in enumerate(train_dataloader):

            image, label = image.to(device), label.to(device)

            optimizer.zero_grad()

            pred = model(image)

            loss = criterion(pred, label)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (image, label) in enumerate(test_dataloader):
            image, label = image.to(device), label.to(device)

            pred = model(image)

            loss = criterion(pred, label)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if label[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            hidden_layer_activation_list.append(model.layer_activations)

        # this conains activations for all epochs 
        final_spareness.append(hidden_layer_activation_list)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

        # if total_test_loss < best_test_loss:
        #     best_test_loss = total_test_loss
            # print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
            # torch.save(model.state_dict(), "model.dth")

    sparseness_list = list()

    for single_epoch_spareness in final_spareness:
        single_epoch_spareness = torch.stack(single_epoch_spareness)
        layer_activations_list = torch.reshape(single_epoch_spareness, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return test_acc, sparseness_list

# AdaDelta

In [ ]:
model_Adadelta = Model()
print("model_Adadelta:", model_Adadelta)
model_Adadelta.to(device)
model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
Adadelta_test_acc, Adadelta_sparseness_list = sparsity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta)

f = open("sorted_sparsity_Adadelta.txt", "w")
f.write(str(Adadelta_test_acc)+'\n'+str(Adadelta_sparseness_list)+'\n\n')

f.close()

!cp sorted_sparsity_Adadelta.txt /content/drive/MyDrive

# AdaGrad

In [10]:
model_Adagrad = Model()
print("model_Adagrad:", model_Adagrad)
model_Adagrad.to(device)
model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
Adagrad_test_acc, Adagrad_sparseness_list = sparsity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad)

f = open("sorted_sparsity_Adagrad.txt", "w")
f.write(str(Adagrad_test_acc)+'\n'+str(Adagrad_sparseness_list)+'\n\n')

f.close()

!cp sorted_sparsity_Adagrad.txt /content/drive/MyDrive

model_Adagrad: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.13042085, Test Loss: 7.76099041, Test Accuracy: 0.10090000

Epoch: 2/100, Train Loss: 0.09620166, Test Loss: 5.50175055, Test Accuracy: 0.10090000

Epoch: 3/100, Train Loss: 0.05869379, Test Loss: 4.44656160, Test Accuracy: 0.11530000

Epoch: 4/100, Train Loss: 0.04409782, Test Loss: 4.14679875, Test Accuracy: 0.20030000

Epoch: 5/100, Train Loss: 0.03957421, Test Loss: 3.77036806, Test Accuracy: 0.25950000

Epoch: 6/100, Train Loss: 0.03484181, Test Loss: 3.69523550, Test Accuracy: 0.26800000

Epoch: 7/100, Train Loss: 0.03376216, Test Loss: 3.49587878, Test Accuracy: 0.29890000

Epoch: 8/100, Train Loss: 0.03238552, Test Loss: 3.25392948, Test Accuracy: 0.32170000

Epoch: 9/100, Train Loss: 0.03214524, Test Loss: 3.15860640, Test Accuracy: 0.35740000

Epoch: 10/100, Train Loss: 

# SGD

In [11]:
model_SGD = Model()
print("model_SGD:", model_SGD)
model_SGD.to(device)
model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
SGD_test_acc, SGD_sparseness_list = sparsity_trainer(optimizer=optimizer_SGD, model=model_SGD)

f = open("sorted_sparsity_SGD.txt", "w")
f.write(str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')

f.close()

!cp sorted_sparsity_SGD.txt /content/drive/MyDrive

model_SGD: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.07253962, Test Loss: 8.02195539, Test Accuracy: 0.10090000

Epoch: 2/100, Train Loss: 0.09554920, Test Loss: 7.73677037, Test Accuracy: 0.10090000

Epoch: 3/100, Train Loss: 0.10922550, Test Loss: 7.40246572, Test Accuracy: 0.10090000

Epoch: 4/100, Train Loss: 0.10637859, Test Loss: 6.74322132, Test Accuracy: 0.10090000

Epoch: 5/100, Train Loss: 0.09017775, Test Loss: 5.89124400, Test Accuracy: 0.10090000

Epoch: 6/100, Train Loss: 0.07428950, Test Loss: 5.22766732, Test Accuracy: 0.10700000

Epoch: 7/100, Train Loss: 0.06468045, Test Loss: 4.79500967, Test Accuracy: 0.13470000

Epoch: 8/100, Train Loss: 0.05956198, Test Loss: 4.49108117, Test Accuracy: 0.16080000

Epoch: 9/100, Train Loss: 0.05663111, Test Loss: 4.25804609, Test Accuracy: 0.18370000

Epoch: 10/100, Train Loss: 0.05

# Adam

In [12]:
model_Adam = Model()
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_sparseness_list = sparsity_trainer(optimizer=optimizer_Adam, model=model_Adam)

f = open("sorted_sparsity_Adam.txt", "w")
f.write(str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')

f.close()

!cp sorted_sparsity_Adam.txt /content/drive/MyDrive

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.99978316, Test Loss: 4.81287643, Test Accuracy: 0.10090000

Epoch: 2/100, Train Loss: 0.55548300, Test Loss: 4.14184658, Test Accuracy: 0.12030000

Epoch: 3/100, Train Loss: 0.20939031, Test Loss: 4.29071359, Test Accuracy: 0.20760000

Epoch: 4/100, Train Loss: 0.13790907, Test Loss: 4.56007608, Test Accuracy: 0.26640000

Epoch: 5/100, Train Loss: 0.12020153, Test Loss: 4.44719206, Test Accuracy: 0.27330000

Epoch: 6/100, Train Loss: 0.10811880, Test Loss: 4.71846524, Test Accuracy: 0.29840000

Epoch: 7/100, Train Loss: 0.09789141, Test Loss: 4.87918231, Test Accuracy: 0.30630000

Epoch: 8/100, Train Loss: 0.08637559, Test Loss: 4.47880004, Test Accuracy: 0.36170000

Epoch: 9/100, Train Loss: 0.08035936, Test Loss: 4.75962087, Test Accuracy: 0.36790000

Epoch: 10/100, Train Loss: 0.0